# LSTM学习笔记

## 目录
1. [LSTM简介](#1-lstm简介)
2. [RNN的局限性回顾](#2-rnn的局限性回顾)
3. [LSTM的核心思想](#3-lstm的核心思想)
4. [LSTM的详细结构](#4-lstm的详细结构)
5. [LSTM的数学原理](#5-lstm的数学原理)
6. [代码实现](#7-代码实现)
7. [练习题](#10-练习题)

## 1. LSTM简介

**LSTM（Long Short-Term Memory）** 是一种特殊的循环神经网络（RNN），由Hochreiter和Schmidhuber在1997年提出。LSTM专门设计用来解决传统RNN的梯度消失问题，能够学习长期依赖关系。

### LSTM的核心优势
- **解决梯度消失问题**：通过门控机制控制信息流
- **长期记忆能力**：能够记住很久之前的信息
- **选择性遗忘**：能够选择性地遗忘不重要的信息
- **灵活的信息更新**：可以决定何时更新记忆

## 2. RNN的局限性

- 梯度消失问题
在传统RNN中，梯度在反向传播过程中会指数衰减：

$$\frac{\partial L}{\partial h_1} = \frac{\partial L}{\partial h_T} \prod_{t=2}^{T} \frac{\partial h_t}{\partial h_{t-1}}$$

当 $\|\frac{\partial h_t}{\partial h_{t-1}}\| < 1$ 时，连乘导致梯度消失。

- 长期依赖问题

- 信息覆盖问题


## 3. LSTM的核心思想

###  细胞状态（Cell State）
LSTM的关键创新是引入了**细胞状态** $C_t$，它像一条传送带，信息可以在其上流动而几乎不发生变化。

### 门控机制（Gate Mechanism）
LSTM通过三个门来控制信息流：
1. **遗忘门（Forget Gate）**：决定从细胞状态中丢弃什么信息
2. **输入门（Input Gate）**：决定什么新信息被存储在细胞状态中
3. **输出门（Output Gate）**：决定输出什么部分的细胞状态

## 4. LSTM的详细结构

###  LSTM单元结构图
![LSTM结构图](img/lstm.png)

###  符号说明
- $h_t$：隐藏状态（输出）
- $C_t$：细胞状态
- $x_t$：当前输入
- $f_t$：遗忘门
- $i_t$：输入门
- $\tilde{C}_t$：候选值
- $o_t$：输出门
- $\sigma$：sigmoid函数
- $\tanh$：双曲正切函数

###  信息流动过程
1. **遗忘阶段**：决定丢弃什么信息
2. **输入阶段**：决定存储什么新信息
3. **更新阶段**：更新细胞状态
4. **输出阶段**：决定输出什么信息

## 5. LSTM的数学原理

### 遗忘门（Forget Gate）
遗忘门决定从细胞状态中丢弃什么信息：

$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

- 输出值在0到1之间
- 1表示"完全保留"
- 0表示"完全丢弃"

###  输入门（Input Gate）
输入门决定什么新信息被存储在细胞状态中：

$$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$

候选值向量：
$$\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$$

###  细胞状态更新
结合遗忘门和输入门来更新细胞状态：

$$C_t = f_t * C_{t-1} + i_t * \tilde{C}_t$$

这个公式是LSTM的核心：
- $f_t * C_{t-1}$：保留旧信息
- $i_t * \tilde{C}_t$：添加新信息

### 输出门（Output Gate）
输出门决定输出什么部分的细胞状态：

$$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$$

隐藏状态（输出）：
$$h_t = o_t * \tanh(C_t)$$

###  完整的LSTM前向传播
```
1. f_t = σ(W_f · [h_{t-1}, x_t] + b_f)     # 遗忘门
2. i_t = σ(W_i · [h_{t-1}, x_t] + b_i)     # 输入门
3. C̃_t = tanh(W_C · [h_{t-1}, x_t] + b_C)  # 候选值
4. C_t = f_t * C_{t-1} + i_t * C̃_t        # 更新细胞状态
5. o_t = σ(W_o · [h_{t-1}, x_t] + b_o)     # 输出门
6. h_t = o_t * tanh(C_t)                   # 输出隐藏状态
```

## 6. 代码实现

In [6]:
import torch
import torch.nn as nn

###  手动实现LSTM单元

In [5]:
class LSTMCell(nn.Module):
    """手动实现的LSTM单元"""
    
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # 遗忘门参数
        self.W_f = nn.Linear(input_size + hidden_size, hidden_size)
        
        # 输入门参数
        self.W_i = nn.Linear(input_size + hidden_size, hidden_size)
        
        # 候选值参数
        self.W_C = nn.Linear(input_size + hidden_size, hidden_size)
        
        # 输出门参数
        self.W_o = nn.Linear(input_size + hidden_size, hidden_size)
        
        # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
    
    def forward(self, x, hidden_state):
        """
        前向传播
        x: 输入 [batch_size, input_size]
        hidden_state: (h_{t-1}, C_{t-1})
        """
        h_prev, C_prev = hidden_state
        
        # 拼接输入和前一时刻的隐藏状态
        combined = torch.cat([x, h_prev], dim=1)  # [batch_size, input_size + hidden_size]
        
        # 1. 遗忘门
        f_t = self.sigmoid(self.W_f(combined))
        
        # 2. 输入门
        i_t = self.sigmoid(self.W_i(combined))
        
        # 3. 候选值
        C_tilde = self.tanh(self.W_C(combined))
        
        # 4. 更新细胞状态
        C_t = f_t * C_prev + i_t * C_tilde
        
        # 5. 输出门
        o_t = self.sigmoid(self.W_o(combined))
        
        # 6. 输出隐藏状态
        h_t = o_t * self.tanh(C_t)
        
        return h_t, C_t
    
    def init_hidden(self, batch_size):
        """初始化隐藏状态"""
        h_0 = torch.zeros(batch_size, self.hidden_size)
        C_0 = torch.zeros(batch_size, self.hidden_size)
        return h_0, C_0

## 7. 练习题

### 理论题

1. **基础概念**
   - 解释LSTM中每个门的作用
   - 为什么LSTM能够解决梯度消失问题？
   - 细胞状态和隐藏状态的区别是什么？

2. **数学推导**
   - 推导LSTM的前向传播公式
   - 解释为什么需要sigmoid和tanh激活函数
   - 计算LSTM相比RNN增加了多少参数

3. **模型对比**
   - 比较LSTM和RNN的优缺点
   - 什么情况下选择双向LSTM？

### 实践题

1. **代码实现**
   - 实现一个GRU单元
   - 修改LSTM模型支持多层和双向
   - 实现一个基于LSTM的序列到序列模型

2. **实验设计**
   - 在长序列数据上比较LSTM和RNN的性能
   - 分析不同隐藏层大小对模型性能的影响
   - 实现并比较不同的LSTM变体

3. **应用拓展**
   - 使用LSTM实现一个简单的机器翻译模型
   - 构建一个基于LSTM的股价预测模型
   - 实现一个LSTM文本分类器

### 思考题
   - 为什么Transformer能够在很多任务上超越LSTM？
   - LSTM在处理什么类型的序列时表现最好？
   - 如何解决LSTM的计算效率问题？
